#Questions
- if one point is have equal distances to multiple segs, how to assign seg to it

In [1]:
import geopandas as gp
from shapely.geometry import LineString, Point
import rtree
import os
import numpy as np

# Functions

build_load_idx(seg_idx_path,segs,update=False): indexing segments

pt2seg(pt,seg_idx): find the segment intersected/nearest to a given point

In [7]:
def build_load_idx(seg_idx_path, segs_with_ids, update=False):
    """
    params:
        seg_idx_path: the path of segment index if the index doesn't exist, the index will be built.
        segs_with_ids: dict: key=objid, value=shapely.geometry.LineString, segments that will be indexed
        update: True: rebuild the index no matter whether the index exists
    Output:
        rtree index of given segments.
    """
    idx_exist = os.path.isfile(seg_idx_path+'.dat') & os.path.isfile(seg_idx_path+'.idx') 
    if update and idx_exist:
        print 'remove existing files'
        os.remove(seg_idx_path+'.dat')
        os.remove(seg_idx_path+'.idx')
        idx_exist = False
    seg_idx = rtree.index.Rtree(seg_idx_path)
    if not idx_exist:
        print 'building idx'
        for objid,seg in segs_with_ids.items():
            seg_idx.insert(objid, seg.bounds)
        print 'built idx'

    return seg_idx

def pt2seg(pt, segs_with_ids, seg_idx):
    """
    params: 
        pt: shapely.geometry.Point
        segs_with_ids:
        seg_idx: rtree index of segments
    Process:
        1. load segment index and find segments(var:idx_match) whose bounds intersect with point's bounds
           if there isn't any matched segment, the point will be buffered until len of idx_match !=0
        2. find the segments that actually intersect with given point
           if there isn't any intersected segment, nearest segment will be returned
    Output:
       hint: whether the returned segid is "nearest" or "intersected"
       segid: assigned segment id
       cnt: len of candidates 
       candidates: the index of segments that are "equally near"/"intersected" to the point
       distance: the distance between point and segment
    """
    
    idx_match = list(seg_idx.intersection(pt.bounds))

    # in case no idx is matched
    bfr_value = 0.000001
    while len(idx_match)==0:
        idx_match = list(seg_idx.intersection(pt.buffer(bfr_value).bounds))
        bfr_value*=2
    
    fil = [pt.intersects(segs_with_ids[idx]) for idx in idx_match]
    seg_match = [idx for (idx,b) in zip(idx_match, fil) if b]
    
    if len(seg_match)==0:
        dis = np.array([pt.distance(segs_with_ids[idx]) for idx in idx_match])
        minimum = dis.min()
        segid = idx_match[dis.argmin()]
        return 'nearest', segid, int(np.where(dis==minimum)[0].shape[0]), np.array(idx_match)[np.where(dis==minimum)[0]],minimum
    else:
        segid=seg_match[0]
        return 'intersected', segid, len(seg_match), np.array(seg_match),0

# Open DC data

- Street segments in DC
- moving violation

In [3]:
dc_segs = gp.read_file('Street_Segments.geojson')
mv_ptns = gp.read_file('Moving_Violations_in_January_2016.geojson')

In [5]:
segs_with_ids = dict(zip(dc_segs.STREETSEGID,dc_segs.geometry))
dc_seg_idx_path = 'dc_seg_idx'
dc_seg_idx = build_load_idx(dc_seg_idx_path, segs_with_ids)

In [8]:
get_seg = mv_ptns.geometry.apply(lambda x: gp.GeoSeries(pt2seg(x, segs_with_ids, dc_seg_idx)))
get_seg.columns=[['hint','seg','len_can','candidates','distance']]
mv_ptns_get_seg = mv_ptns.join(get_seg)

In [9]:
dc_seg_idx.close()

In [14]:
df_mv_gb = mv_ptns_get_seg[['seg']].reset_index().groupby('seg').count().reset_index()
df_mv_gb.columns= ['seg','cnt']
df_mv_gb.shape

(1001, 2)

In [12]:
def gradient_color(percent):
    import numpy as np
    min_color = np.array([255,255,255])
    max_color = np.array([248,105,107])
    return min_color+(max_color-min_color)*percent

dc_segs_with_mvcnt = dc_segs[['STREETSEGID','geometry']].merge(df_mv_gb, left_on = 'STREETSEGID', right_on = 'seg')
dc_segs_with_mvcnt['log_cnt'] = np.log(dc_segs_with_mvcnt.cnt)
dc_segs_with_mvcnt['norm'] = dc_segs_with_mvcnt.log_cnt/dc_segs_with_mvcnt.log_cnt.max()
dc_segs_with_mvcnt['color'] = dc_segs_with_mvcnt.norm.apply(lambda x: '#%02x%02x%02x' % tuple([int(k) for k in gradient_color(x)]))

In [13]:
with open('C:\\Users\\JHW\\Desktop\\data.js', 'w') as f:
    js = dc_segs_with_mvcnt[['geometry','STREETSEGID','color','cnt']].to_json()
    f.write('var segs = %s;' %js)